#

> *Formulate this problem as a Stochastic Programming model. Make sure to specify the first-stage and second-stage decision variables, and to define the objective function and the constraints. Implement the model computationally.*

First we will define the sets that are necessary for problem formulation. There are 2 sets considered:
$$
s = 1, 2, ..., 1000\text{ (The index for scenarios of demand)}
$$
$$
c = 1, 2, 3, 4\text{ (The index for the class of seating, e.g. 1 = economy)}
$$

Next we will define the parameters in the problem
$$
p_{c} = \text{The price of each seating class } c
$$
$$
Pr_{s} = \text{Probability of scenario } s\text{ to occur}
$$
$$
d_{cs} = \text{demand for class } c \text{ in scenario } s
$$

The decision variables of the problem are as follows. There are two decision variables, the first-stage decision variable $a_c$ and the second-stage decision variable $x_cs$
$$
a_{c} = \text{The first-stage integer decision variable for seating class allocation}
$$

$$
x_{cs} = \text{The second-stage integer decision variable for how many seats in each class $c$ was sold in scenario }s
$$

The objective of this problem is to **maximize the expected future revenue**. The expected future revenue is the sum of the expected revenue for all 1000 scenarios revenue(s). In addition, revenue(s) is the sum of the revenue from selling seats in each class $c$. Thus, this is
\begin{equation}
\begin{split}
&\text{max} \sum_{s=1}^{1000}E[\text{revenue}(s)] \\
&= \text{max} \sum_{s=1}^{1000}\left(Pr_{s}\cdot \sum_{c=1}^{4}p_{c}x_{cs}\right)
\end{split}
\end{equation}

The maximization problem is subject to the following constraints

1. The total capacity of the aircraft cannot be gone over the 190 economy seats.
$$
a_1 + 1.2a_2 + 1.5a_3 + 2a_4 \leq 190
$$
2. The number of seats sold for each class $c$ must not go over the assigned capacity $a_c$ for all scenarios $s$
$$
x_{cs} \leq a_{c} \text{                 }\forall c, s
$$
3. The number of seats sold for each class $c$ must not go over the demand for class $c$ for all scenarios $s$
$$
x_{cs} \leq d_{cs} \text{                 }\forall c, s
$$
4. Decision variables are non-negative integers
$$
a_{c} \geq 0 \text{ }\cap \text{ $a_{c} \in$ Z     } \forall c
$$
$$
x_{cs} \geq 0 \text{ }\cap \text{ $x_{cs} \in$ Z      } \forall c, s
$$

The computational implementation of the above follows

In [1]:
# import necessary packages

import numpy as np
from gurobipy import *
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# prepare model parameters and sets

# parameters
p = [400, 500, 800, 1000] # subject to change when incorporating uncertainty here
pr = np.genfromtxt('Pb1_prob.csv', delimiter=',', encoding='utf-8-sig')  # (1000,)
d = np.genfromtxt('Pb1_D_stochastic.csv', delimiter=',', encoding='utf-8-sig')  # (4, 1000)

# define index and sets
scenario_set = range(len(pr))  # s, range(0, 1000)
class_set = range(len(d[:, 0]))  # c, range(0, 4)

In [3]:
# initialize model

model = Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-18


In [4]:
# define the decision variables with their constraints

a = model.addVars(class_set, lb=0, vtype=GRB.INTEGER)  # (40, 1) binary variables
x = model.addVars(class_set, scenario_set, lb=0, vtype=GRB.INTEGER)  # (4, 1000) binary variables

In [5]:
# set up objective function

pb1_objective = sum(pr[s]*sum(p[c]*x[c,s] for c in class_set) for s in scenario_set)
model.setObjective((pb1_objective), GRB.MAXIMIZE)

In [6]:
# setup remaining constraints.

# The total capacity of the aircraft cannot be gone over the 190 economy seats
model.addConstr(a[0] + 1.2*a[1] + 1.5*a[2] + 2*a[3] <= 190)

# The number of seats sold for each class c must not go over the assigned capacity a for all scenarios s
for c in class_set:
    for s in scenario_set:
        model.addConstr(x[c,s] <= a[c])

# The number of seats sold for each class c must not go over the demand for class c for all scenarios s
for c in class_set:
    for s in scenario_set:
        model.addConstr(x[c,s] <= d[c,s])

In [7]:
# Solve the problem

model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 8001 rows, 4004 columns and 12004 nonzeros
Model fingerprint: 0xeb4a9038
Variable types: 0 continuous, 4004 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e-04, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 4888 rows and 888 columns
Presolve time: 0.02s
Presolved: 3113 rows, 3116 columns, 6228 nonzeros
Found heuristic solution: objective 70934.659397
Variable types: 0 continuous, 3116 integer (239 binary)

Root relaxation: objective 7.564067e+04, 2640 iterations, 0.07 seconds (0.21 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It

#

> *Characterize the optimal solution, and the intuition behind it. You may find useful to compute the average demand across all 1,000 scenarios*

The Key metrics from the optimal solution is as follows

In [8]:
# objective value
print("The objective value is: ${} \n".format(model.ObjVal))

# The optimal seating class allocation
print("optimal seating allocation")
print(" economy: {}seats \n economy+: {}seats \n business: {}seats \n first: {}seats \n".format(a[0].x, a[1].x, a[2].x, a[3].x))

# The average demand across all scenarios
print("The average demand for all scenarios")
print(" economy: {}seats \n economy+: {}seats \n business: {}seats \n first: {}seats \n".format(np.mean(d[0, :]), np.mean(d[1, :]), np.mean(d[2, :]), np.mean(d[3, :])))

The objective value is: $75636.12979359292 

optimal seating allocation
 economy: 164.0seats 
 economy+: 15.0seats 
 business: 4.0seats 
 first: 1.0seats 

The average demand for all scenarios
 economy: 180.786seats 
 economy+: 19.558seats 
 business: 6.202seats 
 first: 2.017seats 



Since the optimization here is maximizing the expected value across all scenarios, this could be thought as a *strategy to make most of the average case scenario*. This could be confirmed by comparing the optimal seating allocation and the average seating demand, both which are shown above. It could be confirmed that the two are fairly similar. If we were to take a probability wighted average of the demand scenarios the numbers are expected to be even closer.

#

> *For each seat category, in how many scenarios did you sell fewer seats than the demand, and in how many scenarios did you have excess capacity? Comment briefly.*

We will find such instances using the following code

In [9]:
# scenarios where we sold fewer seats than demand
under_serve_e = 0
under_serve_eplus = 0
under_serve_b = 0
under_serve_f = 0
for s in scenario_set:
    for c in class_set:
        if x[c,s].x < d[c,s]:
            if c == 0:
                    under_serve_e += 1
            if c == 1:
                    under_serve_eplus += 1
            if c == 2:
                    under_serve_b += 1
            if c == 3:
                    under_serve_f += 1
print("scenarios where we under served demand")
print(" economy: {}scenarios \n economy+: {}scenarios \n business: {}scenarios \n first: {}scenarios \n".format(
    under_serve_e, under_serve_eplus, under_serve_b, under_serve_f))

# scenarios where we had excess capacity
over_cap_e = 0
over_cap_eplus = 0
over_cap_b = 0
over_cap_f = 0
for s in scenario_set:
    for c in class_set:
        if d[c,s] < a[c].x:
            if c == 0:
                over_cap_e += 1
            if c == 1:
                over_cap_eplus += 1
            if c == 2:
                over_cap_b += 1
            if c == 3:
                over_cap_f += 1
print("scenarios where we had excess capacity")
print(" economy: {}scenarios \n economy+: {}scenarios \n business: {}scenarios \n first: {}scenarios \n".format(
    over_cap_e, over_cap_eplus, over_cap_b, over_cap_f))


scenarios where we under served demand
 economy: 815scenarios 
 economy+: 731scenarios 
 business: 575scenarios 
 first: 502scenarios 

scenarios where we had excess capacity
 economy: 174scenarios 
 economy+: 211scenarios 
 business: 319scenarios 
 first: 224scenarios 



The results show that the optimal strategy is heavily biased towards limiting excess capacity and trying to have many packed flights. it could be seen that filling all the economy and economy+ seats are key to achieving large profits.

#

> *You are worried that the entry of a low-cost airline will affect prices for the highest classes, and aim to also capture fare uncertainty into your model. You assume that the fares of the four classes will be 400, 500, 800 and 1,000 dollars with 40% probability, 400, 500, 600 and 700 dollars with 30% probability, and 400, 420, 600 and 700 dollars with 30% probability. Demand and price uncertainties are independent.*

> *Define the scenarios and their probabilities carefully. Formulate this problem as a Stochastic Programming model. Implement it computationally.*

With the addition of the 3 uncertainty scenarios in the prices, we now have $1000\times 3 = 3000$ scenarios to consider. Since we assume that the demand and price uncertainties are independent, the probability of each of the 3000 scenarios to occur is the product of the probability of the demand uncertainty and the price uncertainty. Hence, the basic formulation does not change except the addition of another dimension of $n$ for probability $pr$, the price $p$, demand $d$ and decision variable $x$ that now depends on the pricing scenario. If we introduce the set $n$ to represent the set of the new price scenarios the formulation is as follows.

The new sets considered are
$$
s = 1, 2, ..., 1000\text{ (The index for scenarios of demand)}
$$
$$
c = 1, 2, 3, 4\text{ (The index for the class of seatings, e.g. 1 = economy)}
$$
$$
n = 1, 2, 3\text{ (The index for scenarios of price)}
$$


The new parameters in the problem are
$$
p_{cn} = \text{The price of each seating class } c depending on the price scenario $n$
$$
$$
Pr_{sn} = \text{Probability of scenario } s\text{ and $n$ to occur at the same time}
$$
$$
d_{cs} = \text{demand for class } c \text{ in scenario } s
$$

The new decision variables of the problem are as follows.
$$
a_{c} = \text{The first-stage integer decision variable for seating class allocation}
$$

$$
x_{csn} = \text{The second-stage integer decision variable for how many seats in each class $c$ was sold in scenario }s\text{ and $n$ occurring at the same time}
$$

The objective of this problem is the same as in the above problem and to **maximize the expected future revenue**. Thus, with the updated variables above this is
$$
\text{max} \sum_{n=1}^{3}\sum_{s=1}^{1000}\left(Pr_{sn}\cdot \sum_{c=1}^{4}p_{cn}x_{csn}\right)
$$

The new constraints are as follows

1. The total capacity of the aircraft cannot be gone over the 190 economy seats.
$$
a_1 + 1.2a_2 + 1.5a_3 + 2a_4 \leq 190
$$
2. The number of seats sold for each class $c$ must not go over the assigned capacity $a_c$ for all scenarios $s$
$$
x_{csn} \leq a_{c} \text{                 }\forall c, s, n
$$
3. The number of seats sold for each class $c$ must not go over the demand for class $c$ for all scenarios $s$
$$
x_{csn} \leq d_{cs} \text{                 }\forall c, s, n
$$
4. Decision variables are non-negative integers
$$
a_{c} \geq 0 \text{ }\cap \text{ $a_{c} \in$ Z     } \forall c
$$
$$
x_{csn} \geq 0 \text{ }\cap \text{ $x_{csn} \in$ Z      } \forall c, s, n
$$

Now we will implement this in code.

In [10]:
# prepare model parameters and sets

# parameters
p = np.array([[400, 400, 400], [500, 500, 420], [800, 600, 600], [1000, 700, 700]])
d = np.genfromtxt('Pb1_D_stochastic.csv', delimiter=',', encoding='utf-8-sig')  # (4, 1000)
pr = np.genfromtxt('Pb1_prob.csv', delimiter=',', encoding='utf-8-sig')  # (1000,)
pr_1 = pr*0.4
pr_2 = pr*0.3
pr_3 = pr*0.3
pr = np.array([pr_1, pr_2, pr_3]).T  # new pr, (1000, 3)

# define index and sets
scenario_set = range(len(pr))  # s, range(0, 1000)
class_set = range(len(d[:, 0]))  # c, range(0, 4)
price_set = range(3)  # n, range(0, 3)

In [11]:
# initialize model

model = Model()

In [12]:
# define the decision variables with their constraints

a = model.addVars(class_set, lb=0, vtype=GRB.INTEGER)  # (40, 1) binary variables
x = model.addVars(class_set, scenario_set, price_set,  lb=0, vtype=GRB.INTEGER)  # (4, 1000, 3) binary variables

In [13]:
# set up objective function

pb2_objective = sum(sum(pr[s,n]*sum(p[c,n]*x[c,s,n] for c in class_set) for s in scenario_set) for n in price_set)
model.setObjective((pb2_objective), GRB.MAXIMIZE)

In [14]:
# setup remaining constraints.

# The total capacity of the aircraft cannot be gone over the 190 economy seats
model.addConstr(a[0] + 1.2 * a[1] + 1.5 * a[2] + 2 * a[3] <= 190)

# The number of seats sold for each class c must not go over the assigned capacity a for all scenarios s, n
for n in price_set:
    for c in class_set:
        for s in scenario_set:
            model.addConstr(x[c,s,n] <= a[c])

# The number of seats sold for each class c must not go over the demand for class c for all scenarios s, n
for n in price_set:
    for c in class_set:
        for s in scenario_set:
            model.addConstr(x[c,s,n] <= d[c,s])

In [15]:
# Solve the problem

model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 24001 rows, 12004 columns and 36004 nonzeros
Model fingerprint: 0xedea5c74
Variable types: 0 continuous, 12004 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e-04, 8e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 16855 rows and 4855 columns
Presolve time: 0.06s
Presolved: 7146 rows, 7149 columns, 14294 nonzeros
Found heuristic solution: objective 70934.659397
Variable types: 0 continuous, 7149 integer (475 binary)

Root relaxation: objective 7.477592e+04, 4532 iterations, 0.17 seconds (0.41 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Ga

#

> *How many seats did you allocate to each category? Please compare with the solution from the previous model, and comment briefly.*

In [16]:
# report objective value etc.

print("The objective value is: ${} \n".format(model.ObjVal))

# The optimal seating class allocation
print("optimal seating allocation")
print(" economy: {}seats \n economy+: {}seats \n business: {}seats \n first: {}seats \n".format(a[0].x, a[1].x, a[2].x, a[3].x))

# previous optimal allocation
print("previous optimal seating allocation")
print(" economy: {}seats \n economy+: {}seats \n business: {}seats \n first: {}seats \n".format(164, 15, 4, 1))

The objective value is: $74765.16666999667 

optimal seating allocation
 economy: 166.0seats 
 economy+: 15.0seats 
 business: 4.0seats 
 first: 0.0seats 

previous optimal seating allocation
 economy: 164seats 
 economy+: 15seats 
 business: 4seats 
 first: 1seats 



We can see that the optimal seating allocation hardly changes (A first class changed to 2 economy seats). This means that the original seating allocation is a fairly good allocation even in a situation we consider ticket price volatility.